In [ ]:
%pylab inline
import pandas as pd

## Data Loading and Manipulation

In [ ]:
# load data
train_data = pd.read_csv("../input/dataset/train_data.csv")
target = pd.read_csv("../input/dataset/train_target.csv")

# create authors gender dictionary
author_gender = {}
for i in range(len(target)):
    author_gender[target.author[i]] = target.gender[i]
    
# X is the aggregated comments list
X = []
# the genders
y = []
# lengths of X elements
X_len = []
for author, group in train_data.groupby("author"):
    X.append(group.body.str.cat(sep = " "))
    X_len.append([len(group.body)])
    y.append(author_gender[author])

## Preprocessing

In [ ]:
# preprocessing functions
def remove_number(text):
    num = re.compile(r'[-+]?[.\d]*[\d]+[:,.\d]*')
    return num.sub(r'NUMBER', text)

def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'URL',text)

def remove_repeat_punct(text):
    rep = re.compile(r'([!?.]){2,}')
    return rep.sub(r'\1 REPEAT', text)

# remove words that end with one or more identical letters
def remove_elongated_words(text):
    rep = re.compile(r'\b(\S*?)([a-z])\2{2,}\b')
    return rep.sub(r'\1\2 ELONG', text)

def remove_allcaps(text):
    caps = re.compile(r'([^a-z0-9()<>\'`\-]){2,}')
    return caps.sub(r'ALLCAPS', text)

def transcription_smile(text):
    eyes = "[8:=;]"
    nose = "['`\-]"
    smiley = re.compile(r'[8:=;][\'\-]?[)dDp]')
    return smiley.sub(r'SMILE', text)

def transcription_sad(text):
    eyes = "[8:=;]"
    nose = "['`\-]"
    smiley = re.compile(r'[8:=;][\'\-]?[(\\/]')
    return smiley.sub(r'SADFACE', text)

def transcription_heart(text):
    heart = re.compile(r'<3')
    return heart.sub(r'HEART', text)



In [ ]:
import nltk
import re
from nltk.stem import WordNetLemmatizer 
from bs4 import BeautifulSoup   # toglie i tag html
from nltk.corpus import stopwords 
from collections import defaultdict 
from nltk.tokenize import word_tokenize 
from nltk import pos_tag
from nltk.corpus import wordnet as wn
# tags Part of Speech (POS), because teh lemmatizer needs it
tag_map = defaultdict(lambda : wn.NOUN)
# wn does a grammatical analysis
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
# create lemmatizer
word_Lemmatized = WordNetLemmatizer()

def review_to_words(raw_body):
    # remove html tags
    body_text = BeautifulSoup(raw_body).get_text() 
    #letters_only = re.sub("[^a-zA-Z]", " ", body_text)
    # lowercase all text
    words = body_text.lower()
    # remove urls
    text = remove_URL(words)
    # remove numbers
    text = remove_number(text)
    # remove smiles
    text = transcription_sad(text)
    text = transcription_smile(text)
    text = transcription_heart(text)
    text = remove_elongated_words(text)
    words = remove_repeat_punct(text)
    # tokenizes and pass to lemmatizer, which lemmatizes taking tags into account (see before)
    words = word_tokenize(words)
    # we don't remove stop words, because doing it on combination with removing the 40 (trial & error estimated parameter) most utilized words (see below) decreases performance
    #stops = set(stopwords.words("english"))                  
    #meaningful_words = [w for w in words if not w in stops]
    Final_words = []
    for word, tag in pos_tag(words):
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # returns lemmatized texts as strings 
    return( " ".join(Final_words))   

In [ ]:
clean_train_comments = [review_to_words(x) for x in X]

## Train Countvectorizer, Optimize Input for model Training

In [ ]:
#from sklearn.feature_extraction.text import TfidfVectorizer # Il CountVectorizer ha performato meglio
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = "word",
                             max_features = 2000) 
# converts in np array
train_data_features = vectorizer.fit_transform(clean_train_comments).toarray()


print(train_data_features.shape)
# print vocabulary 
vocab = vectorizer.get_feature_names()
print(vocab)
import numpy as np

# counts how many times a word appearco
dist = np.sum(train_data_features, axis=0)

for tag, count in zip(vocab, dist):
    print(count, tag)
    
    # removes the 40 most utilized words
for _ in range(40):
    index = np.argmax(dist)
    train_data_features = np.delete(train_data_features, index, axis = 1)
    
train_data_features.shape

# np array
s = np.concatenate((train_data_features,np.array(X_len)),axis = 1)

# 5000 rows (one per author),  and 2000-40+1 (X_len) features
s.shape

# un np.array
y = np.array(y)

## Train data TruncatedSVD visualization

In [ ]:
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

# plot along the two dimensions with most variance

def plot_LSA(test_data, test_labels, savepath="PCA_demo.csv", plot=True):
        lsa = TruncatedSVD(n_components=2)
        lsa.fit(test_data)
        lsa_scores = lsa.transform(test_data)
        color_mapper = {label:idx for idx,label in enumerate(set(test_labels))}
        color_column = [color_mapper[label] for label in test_labels]
        colors = ['orange','blue']
        if plot:
            plt.scatter(lsa_scores[:,0], lsa_scores[:,1], s=8, alpha=.8, c=test_labels, cmap=matplotlib.colors.ListedColormap(colors))
            orange_patch = mpatches.Patch(color='orange', label='M')
            blue_patch = mpatches.Patch(color='blue', label='F')
            plt.legend(handles=[orange_patch, blue_patch], prop={'size': 20})
            plt.title('BOW + lenght comments only')
            plt.savefig('foo.pdf')

fig = plt.figure(figsize=(8, 8))          
plot_LSA(train_data_features, y)
plt.show()

## Model training and prediction

In [ ]:
from sklearn.model_selection import train_test_split
# splits
X_train, X_valid, y_train, y_valid = train_test_split(s, y,
                                                      train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [ ]:
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# XGBoost model with parameters set with a RandomGridSearch
my_model = XGBRegressor(objective = "reg:logistic",n_estimators=10000, learning_rate=0.01, n_jobs=4,subsample = 0.9,
                       min_child_weight = 1,max_depth=4,gamma=1.5,colsample_bytree=0.6 )
# fits
my_model.fit(X_train, y_train, 
             early_stopping_rounds=80,
             #sample_weight = w,
             eval_set=[(X_valid, y_valid)],
             verbose=False)


In [ ]:
# in the fit function there is the early stop, that one may set iff there is a validation set. The early stop interrupts the training when themodel starts overfitting. But, the model that will predict the test will have no validation during training, so we get here a value and heuristicallly use it also when predicting test
print(my_model.best_iteration)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# cross_val_score resets parameters of my_model and fits it on X_train and t_train with cross validation (we did it for consistency).
kfold = KFold(n_splits=5)
results = cross_val_score(my_model, X_train, y_train, cv=kfold, scoring='roc_auc')

In [ ]:
# print roc of eery fold
results

In [ ]:
# averages
np.mean(results)

In [ ]:
# roc plot
y_score = my_model.predict(X_valid)
# Roc Curve for validation data 
from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(y_valid, y_score)
roc_auc = auc(fpr, tpr) 

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.4f)'% roc_auc )

plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# validation predictions
y_predict = my_model.predict(X_valid)

In [ ]:
from sklearn.metrics import roc_auc_score as roc
# verify
roc(y_valid,y_predict)

In [ ]:
# save predictions
np.save('y_predict_XGB.csv',y_predict)

In [ ]:
# roc evaluated on train set to evaluate overfitting
y_predict = my_model.predict(X_train)
roc(y_train,y_predict)

## Appendix: GridSearch

Thsi is the RandomGridSearch that we used to estimate the best parameters for the XGBoost

```{python}
from xgboost import XGBRegressor
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold

xgb = XGBRegressor(learning_rate=0.01, n_estimators=10000, objective='reg:logistic',
                    silent=True, nthread=1)

#n_esimators si assume indipendente e quindi ottimizzato a parte

 params = {
        'min_child_weight': [1,8],
        'gamma': [0.6,0.8],
        'subsample': [0.9],
        'colsample_bytree': [0.6],
        'max_depth': [4],
        'scale_pos_weight': [1,2.70, 10, 25, 50, 75, 100, 1000]
        }

folds = 5
param_comb = 18 #in realtà di più

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(X_train,y_train), verbose=3, random_state=0)

print('\n Best score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
results.to_csv('xgb-random-grid-search-results-01.csv', index=False)
```